In [1]:
import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)
import requests
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import random
import numpy as np
import pandas as pd
import itertools
from sklearn import svm
from sklearn import tree
from urllib import request
from random import randint
#from wordcloud import WordCloud
from nltk import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
import gensim
from gensim.models import LdaModel, Word2Vec
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
#!pip install mlxtend
#from mlxtend.evaluate import bias_variance_decomp
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hadyo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hadyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hadyo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
books_url_list = ["https://www.gutenberg.org/files/19719/19719-0.txt",
                  "https://www.gutenberg.org/cache/epub/28434/pg28434.txt",
                  "https://www.gutenberg.org/cache/epub/15147/pg15147.txt",
                  "https://www.gutenberg.org/cache/epub/17866/pg17866.txt",
                  "https://www.gutenberg.org/files/3772/3772-0.txt"]

# Preprocessing

In [3]:
#function read raw data
def read_books(url):
    files = []
    for i in url:
        file = requests.get(url = i)
        files.append(file.content.decode('utf-8'))
    return files

#read raw data
raw_dataset = read_books(books_url_list)
len(raw_dataset)

5

In [4]:
#Function cleaned tokens
def cleaned_text(raw_text):
    text_beg = raw_text.find("*** START OF")
    text_end = raw_text.find("*** END OF")

    raw_text = raw_text[text_beg : text_end]
    raw_text = re.findall(r"[a-zA-Z]+", raw_text)
    raw_text = " ".join(raw_text).lower()
    tokens = word_tokenize(raw_text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens
#Getting cleaned tokens
tokens = []
for i in range(len(raw_dataset)):
    tokens.append(cleaned_text(raw_dataset[i]))
len(tokens)

5

In [5]:
#function to extract author name, book name and labels
book_author_mapping = {}
def extract_book_author(raw_text, index):
    label = chr(index + 97)
        
    title_beg = raw_text.find("Title") + 7
    title_end = raw_text.find("\r\n\r\nAuthor")
    title = raw_text[title_beg : title_end]

    author_beg = raw_text.find("Author") + 8
    c = 0
    while raw_text[author_beg + c] != "\r":
        c += 1
        author_end = author_beg + c
    author = raw_text[author_beg : author_end]
    
    book_author_mapping[label] = [title, author]
    
    return label, title, author

#getting the author name, book name, labels
label, author, title = [], [], []
for i in range(len(raw_dataset)):
    label_temp, title_temp, author_temp = extract_book_author(raw_dataset[i],i)
    label.append(label_temp)
    author.append(author_temp)
    title.append(title_temp)
    
print(len(label))
print(len(author))
print(len(title))

5
5
5


In [6]:
#function to partition data
def partitioned_text(tokens, title, author, label):
    final_book_partitions = []
    book_partitions = []
    for j in range(len(tokens)):
        subtractor = len(tokens[j])
        for i in range(0, 200):
            if(subtractor > 150):
                partition = [" ".join(tokens[j][i : i + 150]), title[j], author[j], label[j]]
                book_partitions.append(partition)
                subtractor -= 150
    return book_partitions

In [7]:
#getting partitoins
partitions = pd.DataFrame(partitioned_text(tokens, title, author, label),columns = ['partitions', 'title', 'author', 'label'])

partitions = shuffle(partitions)
partitions

,partitions,title,author,label
99,institute chicago prof john f genung ph amhers...,The Art Of Writing & Speaking The English Lang...,Sherwin Cody,a
513,various branch addition assistance teaching le...,Beeton's Book of Needlework,Isabella Beeton,c
184,effective english among people write advertise...,The Art Of Writing & Speaking The English Lang...,Sherwin Cody,a
767,unpleasant enemy woman instinctively suspected...,Murder in the Gunroom,Henry Beam Piper,d
144,exception chapter iv pronunciation chapter v s...,The Art Of Writing & Speaking The English Lang...,Sherwin Cody,a
...,...,...,...,...
239,name greek letter e g alpha centauri single gr...,The Astronomy of Milton's 'Paradise Lost',Thomas Orchard,b
696,husband collection number people wanted collec...,Murder in the Gunroom,Henry Beam Piper,d
368,v season vi starry heaven vii starry heaven vi...,The Astronomy of Milton's 'Paradise Lost',Thomas Orchard,b
458,material requisite pattern stated chancellor p...,Beeton's Book of Needlework,Isabella Beeton,c


# Feature Engineering

## BOW model

In [8]:
def model_BOW(data): # take the partitions
    cv = CountVectorizer()
    cv_fit = cv.fit_transform(data.iloc[:,0]).toarray()
    cv_fit_df = pd.DataFrame(cv_fit, columns = cv.get_feature_names())
    return cv_fit_df
    

bow_vectorizer = model_BOW(partitions)
bow_vectorizer

,abdegilns,ability,able,absurd,accident,according,accuracy,acid,acknowledged,action,...,write,writer,writing,wrong,wrought,www,year,yet,york,young
0,0,0,0,0,0,0,0,0,1,0,...,2,0,3,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,1,0,...,2,0,0,0,0,0,0,1,0,0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,2,0,2,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
997,0,0,1,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,1
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TF-IDF model

In [9]:
def model_TFIDF(data): # take the partitions
    tfIdf = TfidfVectorizer(use_idf=True)
    tfidf_fit = tfIdf.fit_transform(data.iloc[:,0]).toarray()
    tfidf_fit_df = pd.DataFrame(tfidf_fit, columns = tfIdf.get_feature_names())
    return tfidf_fit_df

tfidf_vectorizer = model_TFIDF(partitions)
tfidf_vectorizer

,abdegilns,ability,able,absurd,accident,according,accuracy,acid,acknowledged,action,...,write,writer,writing,wrong,wrought,www,year,yet,york,young
0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.073503,0.0,...,0.135330,0.000000,0.200378,0.0,0.0,0.0,0.000000,0.075470,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.050595,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.099572,0.065613,0.0,...,0.120804,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.067369,0.000000,0.000000
3,0.0,0.078006,0.000000,0.087929,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.051115,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.073234,0.0,...,0.134834,0.000000,0.133096,0.0,0.0,0.0,0.000000,0.075194,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.047504,0.000000
996,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.057683,0.000000,0.000000,0.000000
997,0.0,0.000000,0.077761,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.156412,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.095921
998,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


## LDA as a feature extractor

In [10]:
def model_LDA2(data): # take the partitions
    cv = CountVectorizer(max_df = 0.9, min_df = 2)
    dtm = cv.fit_transform(data)
    lda = LatentDirichletAllocation(n_components = 5, random_state = 5)
    lda = lda.fit_transform(dtm)
    return lda

lda = model_LDA2(partitions['partitions'])
lda

array([[0.00135697, 0.00135469, 0.00135506, 0.00135817, 0.99457511],
       [0.00132722, 0.00132524, 0.00132523, 0.99469468, 0.00132763],
       [0.00863815, 0.00139554, 0.00136063, 0.10347662, 0.88512907],
       ...,
       [0.00138226, 0.00138097, 0.00138259, 0.00139157, 0.99446261],
       [0.0013348 , 0.00133437, 0.00133488, 0.99465918, 0.00133678],
       [0.00135768, 0.00135432, 0.00135424, 0.00135705, 0.99457673]])

## wordembedding model

In [11]:
partitions_splitted = []
for partition in partitions['partitions']:
    temp = partition.split()
    partitions_splitted.append(temp)


In [12]:
def model_word2vec(tokens_data): # take the partitions_splitted
    model = Word2Vec(sentences = tokens_data, vector_size = 150, workers = 6)
    return model

In [13]:
word2vec_model = model_word2vec(partitions_splitted)
word2vec_model

In [14]:
#Create the word document vectors
def doc_vectors(data_tokens, word2vec_model):
    features = []
    for tokens in data_tokens:
        zero_vector = np.zeros(word2vec_model.vector_size)
        vectors = []
        for token in tokens:
            if token in word2vec_model.wv:
                vectors.append(word2vec_model.wv[token])
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis = 0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

vectorized_doc = doc_vectors(partitions_splitted, word2vec_model)


In [15]:
len(vectorized_doc), len(vectorized_doc[0])

(1000, 150)